# Netatmo data processing

In [ ]:
import glob
from os import path

from uhi_drivers_lausanne import netatmo_utils

## Load multi-variate data from list of JSON responses

In [ ]:
data_dir = "../data/raw/netatmo-lausanne-aug-21"
dst_ts_df_filepath = "../data/interim/netatmo-lausanne-aug-21/ts-df.csv"
dst_station_gser_filepath = "../data/interim/netatmo-lausanne-aug-21/station-gser.gpkg"

In [ ]:
data_filepaths = glob.glob(path.join(data_dir, "*.json"))

# use the following function to load the data from the JSON files into a pandas data
# frame (time-series of measurements) and a geopandas geo-series (station locations)
ts_df, station_gser = netatmo_utils.process_filepaths(data_filepaths)
# drop stations with the same location (likely due to an incorrect set up that led to
# automatic location assignment based on the IP address of the wireless network)
station_gser = station_gser[~station_gser.duplicated(keep=False)]
ts_df = ts_df.loc[:, station_gser.index, :]

  0%|                                                        | 0/310 [00:00<?, ?it/s]/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/m

  9%|████▏                                          | 28/310 [00:00<00:07, 35.97it/s]/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/m

/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
 26%|████████████▏                                  | 80/310 [00:01<00:03, 74.93it/s]/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
 30%|█████████████

 70%|████████████████████████████████              | 216/310 [00:03<00:01, 92.25it/s]/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
 76%|███████████████████████████████████▏          | 237/310 [00:03<00:00, 93.14it/s]/home/martibosch/mambaforge/envs/uhi-drivers-lausanne/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provi

In [ ]:
ts_df.head()

value
variable id                time                      
humidity 70:ee:50:00:10:f0 2021-08-12 13:00:06   50.0
                           2021-08-12 14:00:11   49.0
                           2021-08-12 15:00:16   50.0
                           2021-08-12 16:00:12   49.0
                           2021-08-12 17:00:10   53.0

In [ ]:
station_gser.head()

70:ee:50:00:10:f0    POINT (6.89271 46.52302)
70:ee:50:00:17:8e    POINT (6.90578 46.52483)
70:ee:50:00:20:88    POINT (6.91779 46.56574)
70:ee:50:00:30:2a    POINT (6.68442 46.50841)
70:ee:50:00:36:d0    POINT (6.92119 46.65298)
dtype: geometry

## Dump to files

In [ ]:
ts_df.to_csv(dst_ts_df_filepath)
station_gser.to_file(dst_station_gser_filepath)

ERROR 1: PROJ: proj_create_from_database: Open of /home/martibosch/mambaforge/envs/uhi-drivers-lausanne/share/proj failed
